In [2]:
# Import necessary modules
# glob allows unix style pathname pattern expansion
from glob import glob
# pydicom is the python dicom reader
import pydicom as dicom
import nibabel.nicom.csareader as csareader
import os
import pandas as pd

In [15]:
# 5-T1w_MPR_vNav -- MPRAGE structrual dicoms
# 6-fMRI_DistortionMap_PA -- fmri field map dicoms
# 7-fMRI_DistortionMap_AP -- fmri field map dicoms opposite phase encode direction
# 9-fMRI_REVL_ROI_loc_2 -- fmri localizer dicoms run1
# 10-fMRI_REVL_Study_1 -- fmri task dicoms run1
# 14-dMRI_DistortionMap_AP_dMRI_REVL -- dwi field map dicoms
# 16-dMRI_AP_REVL -- diffusion weighted dicoms

user_ad = 'amattfel'

# directories will have to be specific to your computer
base_dir = f'/home/{user_ad}/Mattfeld_PSB6351'
proj_dir = '/sourcedata'
subj_dir = '/Mattfeld_REVL-000-vCAT-021-S1/scans/10-fMRI_REVL_Study_1/resources/DICOM/files'
#subj_dir = '/Mattfeld_REVL-000-vCAT-021-S1/s*/*/*/*/*'


# with strings I can concatenate them with simple addition lines
data_dir = base_dir + proj_dir + subj_dir

# Here I use glob to grab the dicom files
# Why would I use glob?  What does it give me?
data_files = glob(data_dir + '/*')


In [16]:
len(data_files)

355

In [25]:
# Here I am using the python dicom reader to read in a dicom header
# What should go between the square brackets []
func_vol = dicom.dcmread(data_files[0])

In [42]:
dir(func_vol)
dir(func_vol['AcquisitionTime'])
func_vol['AcquisitionTime'].value

'155343.815000'

In [43]:
# Elements we're interested in
# Repetition Time
# Echo Time
# Acquisition Matrix 
# Flip Angle
# Acquisition Number
# Protocol Name
# Slice times = data_set[0x00191029].value

print('The number of TRs collected equals: {0}'.format(func_vol["Repetition Time"].value))
'''
print('The timing of the TR was: {0} ms'.format(?)
print('The timing of the TE was: {0} ms'.format(?)
print('The flip angle in degrees was: {0}'.format(?)
print(f'The name of the scan was: {whatgoeshere}'
print(f'The phase encode direction was: {whatgoeshere}'
print('COL = AP or PA; ROW = RL or LR')
print('The slice timing was: {0}'.format(?)
print('The field of view - FOV was: {0}'.format(?)

# Why use the following format to access information in the header?
#print(data_set[int('0051100c', 16)].value) # data_set[0x0051100c].value
# Allows us to access information that is private in the dicom header.

# How do I find phase encoding direction information?

csa_str = data_set[int('00291010', 16)].value
csa_tr = csareader.read(csa_str)
print('The phase encode direction was: {0}'.format(?))
print('1 = +; 0 = -')
print('COL, 1 = PA; COL, 0 = AP')
'''

KeyError: "'Repetition Time'"